In [ ]:
import pandas as pd
import numpy as np

import re
import time
import pickle


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
names = ['review', 'Y']
df = pd.read_csv('drive/MyDrive/imdb/train_data.csv', sep=',', names=names)
df.head()

review  \
NaN     Unnamed: 0 review                                              sentiment   
12838.0 12838      In the 3rd installment of Left Behind the maker...   negative   
4772.0  4772       The sun was not shining it was too wet to play ...   negative   
5342.0  5342       I absolutely ADORED this movie as a child and s...   positive   
9333.0  9333       From what I understand Fox was embarrassed they...   negative   

                                                                       Y  
NaN     Unnamed: 0 review                                              Y  
12838.0 12838      In the 3rd installment of Left Behind the maker...  0  
4772.0  4772       The sun was not shining it was too wet to play ...  0  
5342.0  5342       I absolutely ADORED this movie as a child and s...  1  
9333.0  9333       From what I understand Fox was embarrassed they...  0

In [ ]:
df_val = pd.read_csv('drive/MyDrive/imdb/test_data.csv', sep=',', names=names)

In [ ]:
! pip install "tensorflow==1.15.0"
! pip install tensorflow-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
print(tf. __version__)

1.15.0


In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
list_train = [df[i:i+10] for i in range(0,df.shape[0],10)]
list_test = [df_val[i:i+10] for i in range(0,df_val.shape[0],10)]

In [ ]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['review']) for x in list_train]
elmo_test = [elmo_vectors(x['review']) for x in list_test]


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


KeyboardInterrupt: ignored

In [ ]:

elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [ ]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [ ]:
# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("elmo_test_03032019.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  df['Y'],  
                                                  random_state=42, 
                                                  test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

In [ ]:
preds_valid = lreg.predict(xvalid)

In [ ]:
preds_test = lreg.predict(elmo_test_new)